## Data exploration 

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('/kaggle/input/company-bankruptcy-prediction/data.csv')

In [ ]:
pd.Series(df.columns)

In [ ]:
df['Bankrupt?']

In [ ]:
import plotly.express as px

# Create a histogram
fig = px.histogram(df, x='Bankrupt?', title='Distribution of bankrupt Column')

# Show the plot
fig.show()


In [ ]:
df = pd.read_csv('/kaggle/input/company-bankruptcy-prediction/data.csv')

In [ ]:
from sklearn.model_selection import train_test_split

# Features and label
X = df.drop('Bankrupt?', axis=1)
y = df['Bankrupt?']



# Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets+
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y,random_state=42)

# Preprocessing

In [ ]:
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

class OutlierCapper(BaseEstimator, TransformerMixin):
    def __init__(self, lower_percentile=1, upper_percentile=99):
        self.lower_percentile = lower_percentile
        self.upper_percentile = upper_percentile

    def fit(self, X, y=None):
        self.lower_bounds = np.percentile(X, self.lower_percentile, axis=0)
        self.upper_bounds = np.percentile(X, self.upper_percentile, axis=0)
        return self

    def transform(self, X):
        return np.clip(X, self.lower_bounds, self.upper_bounds)

class LogTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return np.log1p(X)

# Initialize the VarianceThreshold object with a threshold of 0
constant_filter = VarianceThreshold(threshold=0)
scaler = StandardScaler()
pca = PCA(n_components=0.95)

# Define the preprocessing pipeline without the PCA step for feature name extraction
preprocessing_pipeline_no_pca = Pipeline(steps=[
    ('outlier_capper', OutlierCapper()),
    ('constant_filter', constant_filter),
    ('log_transform', LogTransformer()),
    ('scaler', scaler)
])

# Fit the pipeline on the training data
preprocessing_pipeline_no_pca.fit(X_train)

# Get the feature names after the scaler step
X_train_processed_no_pca = preprocessing_pipeline_no_pca.transform(X_train) #Rhis is the output without Applying PCA
X_test_processed_no_pca = preprocessing_pipeline_no_pca.transform(X_test)

# The feature names after the scaler step
feature_names_after_scaler = X_train.columns[constant_filter.get_support()]

# print(f'Feature names after scaler step: {feature_names_after_scaler}')



In [ ]:
pd.Series(feature_names_after_scaler)

# Using PCA on Preprocessing for feature Selection

In [ ]:
# Combine steps into a full preprocessing pipeline including PCA
preprocessing_pipeline = Pipeline(steps=[
    ('outlier_capper', OutlierCapper()),
    ('constant_filter', constant_filter),
    ('log_transform', LogTransformer()),
    ('scaler', scaler),
    ('pca', pca)
])

# Fit the full preprocessing pipeline on the training data
preprocessing_pipeline.fit(X_train)

# Transform the training data
X_train_processed = preprocessing_pipeline.transform(X_train)
X_test_processed = preprocessing_pipeline.transform(X_test)
# Get the PCA components
pca_components = pca.components_

# Create a DataFrame for better readability
pca_df = pd.DataFrame(pca_components, columns=feature_names_after_scaler)


In [ ]:
pca_df.head(10)

In [ ]:
no_pca_df = pd.DataFrame(X_train_processed_no_pca, columns=feature_names_after_scaler)

In [ ]:
no_pca_df.head(10)

In [ ]:
no_pca_df_test=pd.DataFrame(X_test_processed_no_pca, columns=feature_names_after_scaler)

In [ ]:
n_pcs = pca.components_.shape[0]

# Get the index of the most important feature on EACH component
most_important = [np.abs(pca.components_[i]).argmax() for i in range(n_pcs)]

# Initial feature names after the scaler step
initial_feature_names = feature_names_after_scaler.copy()

# Get the names of the most important features
most_important_names = [initial_feature_names[most_important[i]] for i in range(n_pcs)]

# Get the explained variance ratio
explained_variance = pca.explained_variance_ratio_

# Create a dictionary with PCA component, most important feature, and explained variance
dic = {'PC{}'.format(i): {'Most Important Feature': most_important_names[i], 'Explained Variance': explained_variance[i]} for i in range(n_pcs)}

# Build the dataframe
df = pd.DataFrame(dic).T



In [ ]:
df.head(10) #pca explaining variance of each column

In [ ]:
# Get the number of components
n_components = pca.n_components_

# Create names for the components
component_names = [f'PC{i+1}' for i in range(n_components)]

# Convert X_train_processed to a DataFrame
X_train_processed_df = pd.DataFrame(X_train_processed, columns=component_names, index=X_train.index)

# Create a DataFrame showing the contribution of each original feature to each principal component
components_df = pd.DataFrame(pca.components_.T, columns=component_names, index=feature_names_after_scaler)

print("Shape of X_train_processed_df:", X_train_processed_df.shape)
# print(X_train_processed_df.head())
components_df.head(10)
#Pca Components for each columns

## Cumulative Variance Explained by pca

In [ ]:
import matplotlib.pyplot as plt

# Calculate cumulative explained variance
cumulative_variance = df['Explained Variance'].cumsum()

# Plotting
plt.figure(figsize=(10, 6))
plt.plot(cumulative_variance, marker='o', linestyle='-', color='b')
plt.title('Cumulative Explained Variance by PCA Components')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.grid(True)
plt.xticks(range(0, len(df)), df.index,rotation=45)
plt.tight_layout()
plt.show()


# Here we can use the approach of Oversampling+Undersampling given by the Research Papers to verify which Sampling technique is best for our dataset  [1](#ref1) [2](#ref2)




Used X_train and y train without preprocessing because the relative difference between all the algorithms wouldn't change much even if we used preprocessed inputs

In [ ]:
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler, EditedNearestNeighbours
from imblearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer, fbeta_score


# Define the techniques to evaluate
oversampling_methods = [SMOTE(sampling_strategy=0.5), ADASYN(sampling_strategy=0.5)]
undersampling_methods = [RandomUnderSampler(sampling_strategy=0.8), EditedNearestNeighbours()]

# Define the model
model = RandomForestClassifier(random_state=42)

# Define the F2 scorer, F2 score is nothing but a variant of F1 score, Higher(2) F2 score weighs more on precision whereas lower(0 to 1) weighs more on recall. Here we are using beta as 2 so
# the overall impact on F2 score will be relatively more determined by precision.
f2_scorer = make_scorer(fbeta_score, beta=2)

# Evaluate each combination
results = []
for over in oversampling_methods:
    for under in undersampling_methods:
        steps = [('over', over), ('under', under)]
        pipeline = Pipeline(steps=steps)
        
        # Apply the pipeline to the training data
        X_temp_resampled, y_temp_resampled = pipeline.fit_resample(X_train, y_train)
        
        # Evaluate the model using cross-validation
        scores = cross_val_score(model, X_temp_resampled, y_temp_resampled, scoring=f2_scorer, cv=5)
        results.append({
            'oversampler': over.__class__.__name__,
            'undersampler': under.__class__.__name__,
            'mean_f2_score': scores.mean()
        })

# Convert results to a DataFrame for easy comparison
results_sample_df = pd.DataFrame(results)
results_sample_df

### Best Technique would be SMOTE + RandomUnderSampling which we already applied

In [ ]:
# Function to get feature names after each step
def get_feature_names_after_pipeline(pipeline, feature_names):
    for name, step in pipeline.named_steps.items():
        if hasattr(step, 'get_feature_names_out'):
            feature_names = step.get_feature_names_out(feature_names)
        elif hasattr(step, 'get_support'):
            feature_names = np.array(feature_names)[step.get_support()]
#         print(f'After {name}: {feature_names}')
    return feature_names

# Assuming original feature names are provided
original_feature_names = X_train.columns  

# Print feature names after each step in the preprocessing pipeline
final_feature_names = get_feature_names_after_pipeline(preprocessing_pipeline, original_feature_names)

In [ ]:
print(f'Final feature names: {final_feature_names}')

In [ ]:
# Get the PCA components

pca_components = pca.components_

# # Create a DataFrame for better readability
# pca_df = pd.DataFrame(pca_components, columns=X_train.columns)
# print(pca_df)

In [ ]:
pca_components.shape

# Check for MultiColinearity, We can use either VIF,Correlation Matrix, Regularization
### Here we will be using ridge regularization to select best features and also remove multicolinearity

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
param_grid = {'alpha': [0.1, 1.0, 10.0, 100.0]}
grid_search = GridSearchCV(estimator=Ridge(), param_grid=param_grid, cv=5)
grid_search.fit(X_train_processed_no_pca, y_train)
best_alpha = grid_search.best_params_['alpha']


In [ ]:
def apply_ridge_regression(X, y, alpha=1.0):
    ridge_reg = Ridge(alpha=alpha)
    ridge_reg.fit(X, y)
    return ridge_reg.coef_
coefficients = apply_ridge_regression(X_train_processed_no_pca, y_train, alpha=best_alpha)  

In [ ]:
regularized_features=pd.DataFrame()

## We take the Middle Threshold here and select all the features lying inside this threshold

In [ ]:
# Assuming X_preprocessed and coefficients are already defined
feature_names = no_pca_df.columns
# Create a DataFrame to store features and their coefficients
regularized_features = pd.DataFrame({
    'Features': feature_names,
    'Coef': coefficients
})

# Sort the DataFrame by coefficient values in descending order of absolute values
# but keep the original sign
regularized_features = regularized_features.reindex(
    regularized_features['Coef'].abs().sort_values(ascending=False).index
)

# Function to select features based on a threshold, considering sign
def select_features(df, threshold):
    return df[(df['Coef'] >= threshold) | (df['Coef'] <= -threshold)]

# Select features with different thresholds
threshold_high = 0.02
threshold_medium = 0.01
threshold_low = 0.005



selected_high = select_features(regularized_features, threshold_high)
selected_medium = select_features(regularized_features, threshold_medium)
selected_low = select_features(regularized_features, threshold_low)

In [ ]:
from IPython.display import display, Markdown

# Print results
display(Markdown('## Feature Coefficients Analysis'))
display(Markdown('Without Caring about the Negative and Positive Values, we just checked the absolute values, i.e., the features which have the most impact on the target.'))

display(Markdown('### Features with |coefficient| >= High Threshold'))
display(selected_high[['Features', 'Coef']])

display(Markdown('### Features with |coefficient| >= Medium Threshold'))
display(selected_medium[['Features', 'Coef']])

display(Markdown('### Features with |coefficient| >= Low Threshold'))
display(selected_low[['Features', 'Coef']])

# Get list of selected features (using medium threshold)
selected_features = selected_medium['Features'].tolist()

X_train_selected = no_pca_df[selected_features]
X_test_selected = no_pca_df_test[selected_features]


# Custom Pipeline classes for preprocessing

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
import pandas as pd

class DataFrameVarianceThreshold(BaseEstimator, TransformerMixin):
    def __init__(self, threshold=0):
        self.threshold=threshold
        self.variance_threshold = VarianceThreshold(threshold=self.threshold)
    
    def fit(self, X, y=None):
        self.variance_threshold.fit(X)
        return self
    
    def transform(self, X):
        X_filtered = self.variance_threshold.transform(X)
        return pd.DataFrame(X_filtered, columns=X.columns[self.variance_threshold.get_support()])
    
class DataFrameScaler(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.scaler = StandardScaler()
    
    def fit(self, X, y=None):
        self.scaler.fit(X)
        return self
    
    def transform(self, X):
        X_scaled = self.scaler.transform(X)
        return pd.DataFrame(X_scaled, columns=X.columns, index=X.index)
    
class DF_log_transform(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # Ensure output is DataFrame with original columns
        return pd.DataFrame(np.log1p(X), columns=X.columns)


class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, coefficients, thresholds):
        self.coefficients = coefficients
        self.thresholds = thresholds
        self.df=df

    def fit(self, X, y=None):
        feature_names = X.columns
        # Create a DataFrame to store features and their coefficients
        self.regularized_features = pd.DataFrame({
            'Features': feature_names,
            'Coef': self.coefficients
        })
        # Sort the DataFrame by coefficient values in descending order of absolute values
        self.regularized_features = self.regularized_features.reindex(
            self.regularized_features['Coef'].abs().sort_values(ascending=False).index
        )
        return self

    def transform(self, X, y=None):
        selected_features = self.select_features(self.regularized_features, self.thresholds)
        return X[selected_features['Features']]

    def select_features(self, df, threshold):
        return df[(df['Coef'] >= threshold) | (df['Coef'] <= -threshold)]


class OutlierCapper(BaseEstimator, TransformerMixin):
    def __init__(self, lower_percentile=1, upper_percentile=99):
        self.lower_percentile = lower_percentile
        self.upper_percentile = upper_percentile

    def fit(self, X, y=None):
        self.lower_bounds = np.percentile(X, self.lower_percentile, axis=0)
        self.upper_bounds = np.percentile(X, self.upper_percentile, axis=0)
        return self

    def transform(self, X):
        X_capped = np.clip(X, self.lower_bounds, self.upper_bounds)
        return pd.DataFrame(X_capped, columns=X.columns, index=X.index)


In [ ]:
# Define the thresholds for regularization coeff
thresholds = {
    'high': 0.02,
    'medium': 0.01,
    'low': 0.005
}

In [ ]:
from sklearn.pipeline import Pipeline

# Define the preprocessing pipeline without the PCA step for feature name extraction
preprocessing_pipeline_final = Pipeline(steps=[
    ('outlier_capper', OutlierCapper()),
    ('constant_filter', DataFrameVarianceThreshold()),
    ('log_transform', DF_log_transform()),
    ('scaler', DataFrameScaler()),  # Use the custom DataFrameScaler
    ('feature_selector', FeatureSelector(coefficients=coefficients, thresholds=thresholds['medium']))
])

# Fit the pipeline on the training data
preprocessing_pipeline_final.fit(X_train)

In [ ]:
# # Transform the training and test data using the updated pipeline
X_train_Final = preprocessing_pipeline_final.transform(X_train)
X_test_Final = preprocessing_pipeline_final.transform(X_test)

In [ ]:
X_train

In [ ]:
from imblearn.combine import SMOTEENN

# Initialize SMOTEENN
smote_enn = SMOTEENN()

# Apply SMOTEENN to the processed training data
X_train_resampled, y_train_resampled = smote_enn.fit_resample(X_train_Final, y_train)

## Visualizing the Distribution of Data in each Feature

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import seaborn as sns

num_features = X_train_Final.shape[1]
num_rows = (num_features - 1) // 6 + 1 
plt.figure(figsize=(20, 5 * num_rows))

for i, feature in enumerate(X_train_Final.columns):
    plt.subplot(num_rows, 6, i + 1)
    sns.histplot(X_train_Final[feature], kde=True, bins=30)
    plt.title(f'{feature}')
    plt.xlabel('')  
    plt.yticks([])  

plt.tight_layout()
plt.show()

# Model Selection and Validation

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, make_scorer, fbeta_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
# from catboost import CatBoostClassifier
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier


In [ ]:
models = {
    'Random Forest': (RandomForestClassifier(random_state=42), {
        'model__n_estimators': [100, 200],
        'model__max_depth': [None, 10, 20],
        'model__class_weight': ['balanced']
    }),
    'Logistic Regression': (LogisticRegression(solver='lbfgs', max_iter=1000, random_state=42), {
        'model__C': [0.001, 0.01, 0.1, 1, 10, 100],
        'model__solver': ['lbfgs', 'saga', 'newton-cg', 'sag'],
        'model__class_weight': ['balanced']
    }),
    'SVM': (SVC(probability=True, random_state=42), {
        'model__C': [0.1, 1, 10],
        'model__kernel': ['linear', 'rbf'],
        'model__class_weight': ['balanced']
    }),
    'AdaBoost': (AdaBoostClassifier(random_state=42), {
        'model__n_estimators': [50, 100],
        'model__learning_rate': [0.01, 0.1, 1]
    }),
    'Gradient Boosting': (GradientBoostingClassifier(random_state=42), {
        'model__n_estimators': [100, 200],
        'model__learning_rate': [0.01, 0.1, 1]
    }),
    'XGBoost': (XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss'), {
        'model__n_estimators': [100, 200],
        'model__learning_rate': [0.01, 0.1, 1]
    }),
    'ANN': (MLPClassifier(random_state=42, max_iter=500, early_stopping=True), {
        'model__hidden_layer_sizes': [(100,), (50, 50)],
        'model__activation': ['relu', 'tanh'],
        'model__solver': ['adam', 'sgd']
    })
}

In [ ]:
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score
import warnings
warnings.filterwarnings('ignore')
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

# Define the F2 scorer
f2_scorer = make_scorer(fbeta_score, beta=2)

# # Split data into training and test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize results dataframe
results_df = pd.DataFrame(columns=['Model', 'Accuracy', 'Precision', 'Recall', 'F1 Score', 'ROC AUC'])

for model_name, (model, param_grid) in models.items():
    print(f"Training {model_name}...")
    
    # Create the pipeline
    pipeline = Pipeline([
        ('model', model)
    ])
    
    # Perform GridSearchCV
    grid_search = GridSearchCV(pipeline, param_grid, scoring=f2_scorer, cv=5, n_jobs=-1)
    grid_search.fit(X_train_resampled, y_train_resampled)
    
    # Predict on the test set
    y_pred = grid_search.predict(X_test_selected)
    scores = grid_search.predict_proba(X_test_selected)[:, 1]
    
    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, scores)
    
    # Append results
    results_df = pd.concat([results_df, pd.DataFrame([{
        'Model': model_name,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'ROC AUC': roc_auc
    }])], ignore_index=True)

    print(f"{model_name} - Done")

# Display results
print(results_df)

# Visualize results
results_df.set_index('Model', inplace=True)
results_df.plot(kind='bar', figsize=(14, 8))
plt.title('Model Comparison')
plt.ylabel('Score')
plt.xticks(rotation=45)
plt.show()


In [ ]:
results_df

In [ ]:
y_train.value_counts()
baseline_predict = [0] * len(y_test)
print("\nBaseline Model:")
print(classification_report(y_test, baseline_predict,zero_division = 0))
print(f"ROC AUC Score: {roc_auc_score(y_test, baseline_predict)}")
print(f"Confusion Matrix:\n{confusion_matrix(y_test, baseline_predict)}")

accuracy = accuracy_score(y_test, baseline_predict)
precision = precision_score(y_test, baseline_predict)
recall = recall_score(y_test, baseline_predict)
f1 = f1_score(y_test, baseline_predict)
roc_auc = roc_auc_score(y_test, baseline_predict)



In [ ]:
results_df = pd.concat([results_df, pd.DataFrame([{
        'Model': 'Baseline Model',
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'ROC AUC': roc_auc
    }]).set_index('Model')])

In [ ]:
results_df

# Plotting every metric

In [ ]:
# import matplotlib.pyplot as plt
# import seaborn as sns
import plotly.offline as pyo
import plotly.express as px

# Activate the offline mode for Plotly
pyo.init_notebook_mode(connected=True)

# Reset the index to turn the model names into a column
results_df.reset_index(inplace=True)
results_df.rename(columns={'index': 'Model'}, inplace=True)

# Melt the DataFrame to long format for Plotly
results_melted = results_df.melt(id_vars='Model', var_name='Metric', value_name='Value')

# Create the plot
fig = px.bar(results_melted, x='Model', y='Value', color='Metric', barmode='group',
             title='Model Performance Metrics',
             labels={'Value': 'Score', 'Model': 'Model'})

# Show the plot in Jupyter notebook
pyo.iplot(fig)

In [ ]:
# Melt the DataFrame to long format for Plotly
results_melted = results_df.melt(id_vars='Model', var_name='Metric', value_name='Value')

# Create the line plot
fig = px.line(results_melted, x='Metric', y='Value', color='Model', markers=True,
              title='Model Performance Metrics',
              labels={'Value': 'Score', 'Metric': 'Metric'})

# Show the plot in Jupyter notebook
pyo.iplot(fig)

In [ ]:
results_df

For now lets focus on XGB and Random Forest we will be stacking them together 

# HyperParameter Tuning Using Optuna and then Stacking the models using LogReg

In [ ]:
import optuna
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, make_scorer, fbeta_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import StackingClassifier


# Random Forest Tuning

In [ ]:
# Suppress Optuna INFO messages
optuna.logging.set_verbosity(optuna.logging.WARNING)

def rf_objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 100, 200)
    max_depth = trial.suggest_int('max_depth', 10, 20)
    model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
    
    model.fit(X_train_resampled, y_train_resampled)
    y_pred_prob = model.predict_proba(X_test_selected)[:, 1]
    
    threshold = trial.suggest_float('threshold', 0.1, 0.9)
    y_pred = (y_pred_prob >= threshold).astype(int)
    
    f1 = f1_score(y_test, y_pred)
    return f1

rf_study = optuna.create_study(direction='maximize')
rf_study.optimize(rf_objective, n_trials=50)
best_rf_params = rf_study.best_params


In [ ]:
best_rf_params

# XGB Tuning

In [ ]:
def xgb_objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 100, 200)
    learning_rate = trial.suggest_loguniform('learning_rate', 0.01, 1.0)
    model = XGBClassifier(n_estimators=n_estimators, learning_rate=learning_rate, random_state=42)
    
    model.fit(X_train_resampled, y_train_resampled)
    y_pred_prob = model.predict_proba(X_test_selected)[:, 1]
    
    threshold = trial.suggest_float('threshold', 0.1, 0.9)
    y_pred = (y_pred_prob >= threshold).astype(int)
    
    f1 = f1_score(y_test, y_pred)
    return f1

xgb_study = optuna.create_study(direction='maximize')
xgb_study.optimize(xgb_objective, n_trials=50)
best_xgb_params = xgb_study.best_params


In [ ]:
best_xgb_params

In [ ]:
rf_model = RandomForestClassifier(
    n_estimators=best_rf_params['n_estimators'],
    max_depth=best_rf_params['max_depth'],
    random_state=42
)

xgb_model = XGBClassifier(
    n_estimators=best_xgb_params['n_estimators'],
    learning_rate=best_xgb_params['learning_rate'],
    random_state=42
)

stacked_model = StackingClassifier(
    estimators=[
        ('rf', rf_model),
        ('xgb', xgb_model)
    ],
    final_estimator=LogisticRegression()
)

# Train the stacked model
stacked_model.fit(X_train_resampled, y_train_resampled)

# Predict with optimized thresholds (use a common threshold for simplicity)
threshold = max(best_rf_params['threshold'], best_xgb_params['threshold'])
y_pred_prob = stacked_model.predict_proba(X_test_selected)[:, 1]
y_pred = (y_pred_prob >= threshold).astype(int)

# Evaluate the stacked model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_prob)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"ROC AUC: {roc_auc}")


# Lets study the Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Predict with optimized threshold
y_pred_prob = stacked_model.predict_proba(X_test_Final)[:, 1]
y_pred = (y_pred_prob >= threshold).astype(int)

# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = conf_matrix.ravel()

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_prob)

# Print the confusion matrix and metrics
print("Confusion Matrix:")
print(conf_matrix)
print(f"True Negatives (TN): {tn}")
print(f"False Positives (FP - Type I Error): {fp}")
print(f"False Negatives (FN - Type II Error): {fn}")
print(f"True Positives (TP): {tp}")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"ROC AUC: {roc_auc}")


In [ ]:
from IPython.display import display, Markdown

# Create the markdown string with variables
markdown_content = f"""
### Confusion Matrix
- **True Negatives (TN):** {tn}
  - The model correctly identified {tn} instances as non-bankrupt (0).

- **False Positives (FP - Type I Error):** {fp}
  - The model incorrectly classified {fp} non-bankrupt instances as bankrupt (1).

- **False Negatives (FN - Type II Error):** {fn}
  - The model incorrectly classified {fn} bankrupt instances as non-bankrupt (0).

- **True Positives (TP):** {tp}
  - The model correctly identified {tp} instances as bankrupt (1).

### Performance Metrics
- **Accuracy:** {accuracy:.4f}
  - Accuracy is the proportion of true results (both true positives and true negatives) among the total number of cases examined. It indicates that the model correctly classified {accuracy:.4f}% of the instances.

- **Precision:** {precision:.4f}
  - Precision is the proportion of true positive results in the predicted positive results. Here, {precision:.4f}% of the instances that the model classified as bankrupt were actually bankrupt. This is relatively low, indicating that there are many false positives.

- **Recall (Sensitivity):** {recall:.4f}
  - Recall is the proportion of true positive results in the actual positive results. The model correctly identified {recall:.4f}% of the bankrupt instances. This is quite good, indicating that the model is effective at catching most of the bankrupt instances.

- **F1 Score:** {f1:.4f}
  - The F1 score is the harmonic mean of precision and recall. It balances precision and recall, and here it is {f1:.4f}, indicating moderate performance in both aspects.

- **ROC AUC:** {roc_auc:.4f}
  - The ROC AUC score is a measure of how well the model distinguishes between the classes. A score of {roc_auc:.4f} indicates that the model has a high ability to distinguish between bankrupt and non-bankrupt instances.
"""

# Display the markdown content
display(Markdown(markdown_content))





### Interpretation
- **High Accuracy and ROC AUC:**
  - The model performs well overall, correctly classifying most instances and effectively distinguishing between bankrupt and non-bankrupt cases.

- **High Recall, Low Precision:**
  - The model has a high recall, meaning it successfully identifies most of the bankrupt cases. However, the precision is low, indicating that many of the instances it identifies as bankrupt are actually non-bankrupt. This results in a higher number of false positives .

- **False Positives vs. False Negatives:**
  - There are more false positives than false negatives . This means that the model is more likely to incorrectly classify a non-bankrupt instance as bankrupt than the other way around. In the context of bankruptcy prediction, this may be a cautious approach, ensuring that potential bankruptcies are flagged, even at the cost of some false alarms.

- **Balancing Precision and Recall:**
  - Given the importance of detecting bankruptcies (recall), the model's current threshold is achieving a good balance. However, depending on your application's tolerance for false positives, you might want to further adjust the threshold or employ additional strategies to improve precision without significantly impacting recall.

# Conclusion
- The model is good at identifying bankrupt instances (high recall), but there is a trade-off with precision, resulting in a notable number of false positives. The high ROC AUC indicates strong overall performance, suggesting that the model is well-calibrated for distinguishing between the classes. Further fine-tuning of the threshold or additional techniques might be employed to improve precision if false positives are a concern

# Downloading the model locally

In [ ]:
import joblib
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

class CombinedModel:
    def __init__(self, preprocessing_pipeline, model, threshold=threshold):
        self.preprocessing_pipeline = preprocessing_pipeline
        self.model = model
        self.threshold = threshold
        self.data_storage = []  # To store incoming data for retraining

    def predict(self, X):
        X_processed = self.preprocessing_pipeline.transform(X)
        y_pred_prob = self.model.predict_proba(X_processed)[:, 1]
        return (y_pred_prob >= self.threshold).astype(int)

    def predict_proba(self, X):
        X_processed = self.preprocessing_pipeline.transform(X)
        return self.model.predict_proba(X_processed)

    def save(self, filepath):
        with open(filepath, 'wb') as f:
            joblib.dump(self, f)

    @staticmethod
    def load(filepath):
        with open(filepath, 'rb') as f:
            return joblib.load(f)

    def store_data(self, X, y):
        self.data_storage.append((X, y))

    def get_stored_data(self):
        X_data = np.concatenate([data[0] for data in self.data_storage], axis=0)
        y_data = np.concatenate([data[1] for data in self.data_storage], axis=0)
        return X_data, y_data



## Same Pipeline as we created before

In [ ]:
preprocessing_pipeline_final = Pipeline(steps=[
    ('outlier_capper', OutlierCapper()),
    ('constant_filter', DataFrameVarianceThreshold(threshold=0)),
    ('log_transform', DF_log_transform()),
    ('scaler', DataFrameScaler()),
    ('feature_selector', FeatureSelector(coefficients=coefficients, thresholds=thresholds['medium']))
])

# Fit the pipeline on the training data
preprocessing_pipeline_final.fit(X_train)




In [ ]:
# Initialize the combined model with the fitted model and preprocessing pipeline
combined_model = CombinedModel(preprocessing_pipeline=preprocessing_pipeline_final, model=stacked_model, threshold=threshold)

# Save the combined model
combined_model.save('combined_model_test.pkl')

In [ ]:
# Load the combined model
loaded_model = CombinedModel.load('combined_model_test.pkl')
new_data=X_test.copy()

predictions = loaded_model.predict(new_data)
print(predictions)




<a id="ref3"></a>
# Interpreting Original Data with Statistics 

In [ ]:
from scipy import stats
import statsmodels.api as sm

X, y = X.align(y, join='inner', axis=0)

X = sm.add_constant(X)  # Add a constant term to the model for mathematical convenience

# Fit the OLS model
model = sm.OLS(y, X).fit()

# Create a summary dataframe
summary_df = pd.DataFrame({
    'Variable': X.columns,
    'Coefficient': model.params,
    'Std. Err.': model.bse,
    't_statistic': model.tvalues,
    'P>|z|': model.pvalues,
    '[0.025': model.conf_int()[0],
    '0.975]': model.conf_int()[1]
})

# Sort by p-value
summary_df = summary_df.sort_values('P>|z|')

# Print the summary
summary_df



## The p value here if less than significance level is basically confirming that the variable/feature has large impact on the target but on the other hand it doesn't mean that other feature/variables have no impact

In [ ]:
# Set significance level
alpha = 0.05
summary_df['Significance'] = summary_df['P>|z|'].apply(lambda p: 'Reject H0' if p < alpha else '    Failed to reject H0')

In [ ]:
summary_df=summary_df.reset_index().drop('index',axis=1)
# Optionally, save to CSV
summary_df.to_csv('financial_ratios_analysis.csv', index=False)

In [ ]:
summary_df

In [ ]:
significant_summary_df=summary_df.loc[1:10]
Descriptive_Stats=X[significant_summary_df.Variable].describe().T
Descriptive_Stats['Skew']=X[significant_summary_df.Variable].skew()
Descriptive_Stats['Kurtosis']=X[significant_summary_df.Variable].kurtosis()

In [ ]:
Descriptive_Stats.to_csv('Descriptive_Stats.csv', index=False)

In [ ]:
Descriptive_Stats

### High kurtosis means basically thick tail and high peak, this can be indication of large majority of companies inside the common range and some of the countries at pretty high or low, In this data if we look at After-tax net Interest Rate and Pre-tax net Interest Rate, The Net Interest Rate is generally interest gained - interest paid after doing Revenue - (COGS+ Other Expenses), The quantity means that most of the companies had the similar range of net Interest rate but there were big amount of outliers which had very low or very high of this interest rate

### Less than 3 Kurtosis means the distribution is moving towards Uniform distribution, Cash/Total Assets,Asset TurnOver and Current Assets/Total assets are almost uniform in most companies. Current Assets are Assets valuated or kept for low time period, usually less than 1 year like liquidity,Credit,deposits,Bonds and other financial instruments. Most of these companies in out data have equivalent amount of Current Assets/Total Assets which should make sense because most of the companies like to keep a specific ratio of fixed and current assets.


In [ ]:
correlation_matrix=X[significant_summary_df.Variable].corr()

In [ ]:
correlation_matrix

In [ ]:
# np.triu(correlation_matrix, k=1)

In [ ]:
# Create a mask for the upper triangle
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
# Set up the matplotlib figure
plt.figure(figsize=(10, 8))
# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(correlation_matrix, mask=mask, annot=True, cmap='coolwarm', vmax=1, vmin=-1, center=0, square=True, linewidths=.5)
# Show the plot
plt.show()


## High correlation between After-Tax and Before-Tax interest rate is expected because 
The after-tax interest rate is calculated as: $After\_Tax\_Interest\_Rate = Before\_Tax\_Interest \cdot (1 - Tax\_Rate)$

## High correlation between Borrowing Dependency and Current Liability to Equity
This also makes sense because the debt is considered as Liability and if u have more debt you have to pay back the debt within a year i.e increase of current Liability.

## Net Income to StockHolder's Equity is inversely proportional to Borrowing Dependency and CL/Equity
This relationship means the Net Income which is nothing but Assets - Liabilities in balance sheet or Revenue-COGS-Operating Costs-Interest-Taxes(Basically all the obligations of a company)/Shareholder's Equity(This is nothing but Equity which the Company carried with it from initial to current stage) will increase when The residual profit will increase(increase of profit margin) which will also make the Company reinvest or pay dividends to its shareholders or buy back stocks, This in short means a good healthy company.
if that is the case then the company is not in debt and even if it is, it has the capability to pay back the debt whereas if this quantity is very low compared to Equity which the company holds as its core, then there are high chances of it borrowing Debt more often.
